## Head
Author: Eli Hecht
Purpose: adapt Alina's sentence embedding code to group decision-study responses

## Imports

In [ ]:
import os

# 
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
import pandas as pd
import json

In [ ]:
# Load the model

from sentence_transformers import SentenceTransformer #load the model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')


In [ ]:
import umap
from sklearn.cluster import KMeans
from scipy.spatial import distance_matrix

In [ ]:
## filtering for personal words

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
# import kaleido
import plotly.io as pio
import matplotlib.pyplot as plt

## Define analysis functions

In [ ]:
def make_directories(dir):
    if not os.path.exists(dir):
        os.mkdir(dir)
    if not os.path.exists(dir + 'plots/'):
        os.mkdir(dir + 'plots/')
    if not os.path.exists(dir + 'cluster_tables/'):
        os.mkdir(dir + 'cluster_tables/')

In [ ]:
# A function to filter specified words and pronouns

def filter_words(sentence, words_to_remove):
    # Tokenize
    words = word_tokenize(sentence)
    
    # Tag each word with its part of speech
    pos_tags = nltk.pos_tag(words)
    
    #Define the pos tags for personal words
    pronoun_tags = {'PRP', 'PRP$'}
    
    words_to_remove_lower = [noun.lower() for noun in words_to_remove]

    #Filter out words that are personal words
    filtered_words = [word for word, tag in pos_tags if tag not in pronoun_tags and word.lower() not in words_to_remove_lower]
    
    #Reassemble the sentence
    return ' '.join(filtered_words)

In [ ]:
# testing pronoun removal
filter_words('my name is Eli. I would go to the park', ["Eli", "would"])

In [ ]:
# function to compute embeddings for a single vignette

def compute_embeddings(df, text_column, vignette_number, words_to_remove):
    options_df = df[df['context'] == vignette_number] 
    options_df = options_df.reset_index()

    # Convert all responses to strings
    options_df[text_column] = options_df[text_column].astype(str)

    # Delete proper name for clustering ease
    options_df[text_column] = options_df[text_column].apply(lambda x: filter_words(x, words_to_remove))

    options = options_df[text_column].tolist()
    options_embeddings = model.encode(options, show_progress_bar=True) # compute option embeddings
    return options_embeddings, options_df



In [ ]:
# dimensionality reduction with pre-defined parameters
def compute_clusters(options_embeddings, clustering_params, umap_params):
    clustering_model = KMeans(n_clusters=clustering_params['num_clusters'], n_init='auto')
    umap_embeddings = (umap.UMAP(n_neighbors=umap_params['n_neighbors'], 
                                    n_components=umap_params['n_components'], 
                                    metric=umap_params['metric'],
                                    min_dist=umap_params['min_dist'],
                                    random_state=umap_params['random_state'])
                                .fit_transform(options_embeddings))
    
    # perform KMeans clustering and look at clusters 
    clustering_model.fit(umap_embeddings)
    return umap_embeddings, clustering_model.labels_, clustering_model



In [ ]:
def print_clusters(options_df, text_column_name, cluster_assignment, numb_clusters):
    clustered_sentences = [[] for i in range(numb_clusters)]
    for sentence_id, cluster_id in enumerate(cluster_assignment):
        clustered_sentences[cluster_id].append(options_df[text_column_name][sentence_id])

    for i, cluster in enumerate(clustered_sentences):
        print("Cluster ", i+1)
        cluster_list = []
        for item in cluster:
            # removes identical context that is included in text of each result
            cluster_list.append(item.split(":", 1)[-1].strip())
        print(cluster_list)
        print("")

In [ ]:
def k_plot(options_embeddings, umap_params, directory):
    distortions = []
    K_range = range(1, 18)


    umap_embeddings = (umap.UMAP(n_neighbors=umap_params['n_neighbors'], 
                                        n_components=umap_params['n_components'], 
                                        metric=umap_params['metric'],
                                        min_dist=umap_params['min_dist'],
                                        random_state=umap_params['random_state'])
                                    .fit_transform(options_embeddings))
    
    # perform KMeans clustering and look at clusters 
    for k in K_range:
        # print("k:" + str(k))
        clustering_model = KMeans(n_clusters=k, n_init='auto')
        clustering_model.fit(umap_embeddings)
        distortions.append(clustering_model.inertia_)

    # Plotting the elbow curve
    plt.plot(K_range, distortions, marker='o')
    plt.title('Elbow Method For Optimal k')
    plt.xlabel('Number of Clusters (k)')
    plt.ylabel('Sum of Squared Distances')
    plt.savefig(directory)
    plt.show()

In [ ]:
# create a df with clusters, centroids and umap embeddings
def compute_centroid(options_df, text_column_name, cluster_assignment, clustering_model, umap_embeddings):
    # attach the cluster assignments to the dataframe
    options_df['cluster'] = pd.Series(cluster_assignment, index=options_df.index)

    # Attach the centroids to the dataframe
    # In sklearn, the cluster centers are available directly via clustering_model.cluster_centers_
    options_df['centroid'] = options_df['cluster'].apply(lambda x: clustering_model.cluster_centers_[x])

    # Split the UMAP embeddings into individual columns for easier processing later
    for i in range(umap_embeddings.shape[1]):
        options_df[f'umap_dim_{i}'] = umap_embeddings[:, i]

    # Convert the UMAP embeddings from individual columns to lists for use in the distance calculation
    umap_list = [f'umap_dim_{i}' for i in range(umap_embeddings.shape[1])]
    options_df['umap_embedding_list'] = options_df[umap_list].apply(lambda row: row.tolist(), axis=1)

    # Define a function to compute the distance of each embedding from its cluster's centroid
    def distance_from_centroid(row):
        return distance_matrix([row['umap_embedding_list']], [row['centroid']])[0][0]
    options_df['distance_from_centroid'] = options_df.apply(distance_from_centroid, axis=1)

    # Select the 'response' closest to each cluster centroid to serve as a summary of the cluster
    summary = options_df.sort_values('distance_from_centroid', ascending=True).groupby('cluster').head(1).sort_index()[text_column_name].tolist()

    # Create a dictionary linking each summary 'response' to its corresponding cluster number
    clusters = {}
    for i in range(len(summary)):
        clusters[summary[i]] = options_df.loc[options_df[text_column_name] == summary[i], "cluster"].iloc[0]

    return clusters

In [ ]:
def umap_2(options_df, options_embeddings, clusters, umap_params):
    # umaps to 2 dimensions for plotting
    umap_embeddings_2 = (umap.UMAP(n_neighbors=umap_params['n_neighbors'], 
                                    n_components=2, 
                                    metric=umap_params['metric'],
                                    min_dist=umap_params['min_dist'],
                                    random_state=umap_params['random_state'])
                                .fit_transform(options_embeddings))
    for i in range(umap_embeddings_2.shape[1]):
        options_df[f'umap_dim2_{i}'] = umap_embeddings_2[:, i]

    # create a dictionary with cluster numbers as keys and their centroid options as values
    id_to_name = {v: k for k, v in clusters.items()}

    options_df['cluster_name'] = options_df['cluster'].map(id_to_name) # create a column with centroid options

    options_df['cluster_name'] = options_df['cluster_name'].apply(lambda x: x.split(":", 1)[-1].strip())

In [ ]:
# Function to plot clusters on the joint embedding space


# def normalize(values):
#     min_val = np.min(values)
#     max_val = np.max(values)
#     if max_val - min_val == 0:  # Avoid division by zero if all values are the same
#         return np.ones_like(values)
#     return (values - min_val) / (max_val - min_val)
# 3:15

def plot_clusters(component1, component2, cluster, name, response,
                   data_source, 
                   umap_params, clustering_params,
                   plot_location):
    pio.renderers.default = "browser"
    color_palette = px.colors.qualitative.Light24
    
    fig = go.Figure()
    
    title_str = f"UMAP Parameters: n_neighbors={umap_params['n_neighbors']}, n_components={umap_params['n_components']}, min_dist={umap_params['min_dist']} | Clustering: {clustering_params['algorithm_name']} with k={clustering_params['num_clusters']} clusters"
    
    # Get unique clusters and data sources
    unique_clusters = sorted(cluster.unique())
    unique_data_sources = sorted(data_source.unique())

    color_map = {uc: color_palette[i % len(color_palette)] for i, uc in enumerate(unique_clusters)}
    marker_symbols = {
        unique_data_sources[1]: 'square',
        unique_data_sources[0]: 'diamond'
    }
    
    # # Define colors for each trajectory
    # trajectory_colors = {
    #     unique_data_sources[0]: 'red',
    #     unique_data_sources[1]: 'blue'
    # }
    
    # Add a trace for each Source to indicate the shape in the legend
    for ds, symbol in marker_symbols.items():
        fig.add_trace(go.Scatter(
            x=[None],
            y=[None],
            mode='markers',
            marker=dict(
                size=10,
                symbol=symbol,
                # color=trajectory_colors[ds]
            ),
            name=f'{ds.capitalize()} (shape and trajectory)'
        ))

    added_cluster_names = set()
    for ds in unique_data_sources:
        for uc in unique_clusters:
            mask = (cluster == uc) & (data_source == ds)
            if mask.any():  # Check if there are any rows after applying the mask
                # normalized_opacities = 1 - normalize(gen_num[mask])
                # opacity_values = 0.2 + 0.8 * normalized_opacities
                # opacity_values = np.nan_to_num(opacity_values) 
                show_in_legend = name[mask].iloc[0] not in added_cluster_names
                # show_in_legend = False
                added_cluster_names.add(name[mask].iloc[0])
                
                fig.add_trace(go.Scatter(
                    x=component1[mask],
                    y=component2[mask],
                    mode='text+markers',
                    name=name[mask].iloc[0] if show_in_legend else None,
                    legendgroup=f'group{uc}',
                    showlegend=show_in_legend,
                    hovertext= str(uc) + ": " + response[mask] ,
                    # text='gen_num[mask].astype(str)',
                    marker=dict(
                        size=12,
                        color=color_map[uc],
                        symbol=marker_symbols[ds],  # Use the marker symbol based on the data source
                        line_width=1,
                        opacity=1
                    ),
                    textfont=dict(
                        size=10,
                        color='black'
                    )
                ))


    fig.update_layout(
        margin=dict(l=100, r=100, b=100, t=100),
        width=2000,
        height=1200,
        showlegend=False,
        title=title_str,
        paper_bgcolor='white',  # White background for the entire plot area
        plot_bgcolor='white',
        legend=dict(
            yanchor="top",
            y=1,
            xanchor="left",
            x=0.01,
            bgcolor='rgba(255,255,255,0)'
        )
    )

    fig.layout.template = 'ggplot2'
    fig.write_html(plot_location)

## Load in data

In [ ]:
agent_list = ['Heinz', 'Josh', 'Brian', 'Liz', 'Mary', 'Brad', 'Darya', 'Eunice', 'Eamon', 'Cameron', 'Erica', 'Carl', 'Daniel', 'Andy', 'Ahmed', 'Eva', 'Jeff', 'Shania']

In [ ]:
df_decision = pd.read_csv('../data/decision.csv')

# add id column
df_decision = df_decision.reset_index().rename(columns={'index': 'id'})
df_decision['id'] += 1

# select only participants who finished
df_decision = df_decision[df_decision['finished']]

# exclude ids of participants who gave non-sensical responses
exclude_ids = [21, 64, 72, 74, 84, 86, 89]
df_decision[~df_decision['id'].isin(exclude_ids)].reset_index(drop=True) 

# Select columns 'id', 'S1_1' to 'S18_1'
df_decision = df_decision[['id', 'S1_1', 'S2_1', 'S3_1', 'S4_1', 'S5_1', 'S6_1', 'S7_1', 'S8_1', 'S9_1', 'S10_1', 'S11_1', 'S12_1', 'S13_1', 'S14_1', 'S15_1', 'S16_1', 'S17_1', 'S18_1']]

#  Melt the DataFrame to long format
df_decision = pd.melt(df_decision, id_vars=['id'], var_name='context', value_name='decision')

# Extract numeric values from 'context' column using str.extract
df_decision['context'] = df_decision['context'].str.extract('(\d+)').astype(int)


df_decision.dropna(subset=['decision'], inplace=True)
df_decision.rename(columns={'decision': 'response'}, inplace=True)

# add source column
df_decision['source'] = 'decision'

# df_decision

In [ ]:
df_pg = pd.read_csv('../manualCoding/pg_coded_final.csv', index_col=0)
df_pg = df_pg[['context', 'id', 'answer', 'text', 'value']]
df_pg.rename(columns={"text":"response"}, inplace=True)
df_pg['source'] = 'pg'

In [ ]:
# Merge decision study data and possibility generation study data into one data frame for clustering
df = pd.merge(df_decision, df_pg, how='outer')

In [ ]:
# Add full scenario texts to merged_text to give LLM context for responses
contexts_list = pd.read_csv('../materials/contextsTable.csv', index_col=0)['text']

df_merge = pd.merge(df, contexts_list, left_on='context', right_index=True)
df_merge['merged_text'] = df_merge['text'] + ' : ' + df_merge['response']
df_merge.rename(columns={"response": "response_original", "text": "scenario_text"}, inplace=True)

df = df_merge

In [ ]:
df

## Compute VRC for each scenario to determinal optimal k

In [ ]:
from sklearn.metrics import calinski_harabasz_score


# text_column_name options: 'response_original' or 'merged_text'
# merged_text gives context by merging response with the scenario text and is preferred
text_column_name = 'merged_text'


# directory to place resulting plots and clusters
dir = "numVRC/"

# check that path exists and make it if it doesn't
if not os.path.exists(dir + 'elbow_plots/'):
    if not os.path.exists(dir):
            os.mkdir(dir)
    os.mkdir(dir + 'elbow_plots/')

# Define UMAP and clustering parameters
umap_params = {'n_neighbors': 100, 'n_components': 10, 'metric': 'cosine', 'min_dist': 0.05, 'random_state': None}

def compute_VRC(embeddings, umap_params):
    umap_embeddings = (umap.UMAP(n_neighbors=umap_params['n_neighbors'], 
                             n_components=umap_params['n_components'], 
                             metric=umap_params['metric'],
                             min_dist=umap_params['min_dist'],
                             random_state=umap_params['random_state'])
                             .fit_transform(embeddings))
     
    # Define a range of k values to try
    k_values = range(2, 20)  # You can adjust this range as needed
    # Initialize an empty list to store the Calinski-Harabasz scores
    ch_scores = []

    # perform KMeans clustering and look at clusters 
    for k in k_values:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init = 'auto')
        kmeans.fit(umap_embeddings)
        labels = kmeans.labels_
        ch_score = calinski_harabasz_score(umap_embeddings, labels)
        ch_scores.append(ch_score)


    # Find the optimal k that maximizes the Calinski-Harabasz index
    optimal_k = k_values[np.argmax(ch_scores)]

    # print(f"Optimal number of clusters (k): {optimal_k}")
    return optimal_k

vrc_results = {}
# creates plots and cluster_tables for each scenario
for scenario_number in range(1, 19):
    print("Running analysis on scenario " + str(scenario_number))
    # selects appropriate agent name based on scenario number
    agent_name = agent_list[scenario_number-1]
    words_to_remove = [agent_name, 'should', 'would', 'could']

    options_embeddings, options_df = compute_embeddings(df, text_column_name, scenario_number, words_to_remove)

    # Compute Variance Ratio Criterion to determine appropriate number of clusters
    optimal_k = compute_VRC(options_embeddings, umap_params)
    vrc_results[str(scenario_number)] = optimal_k
   


In [ ]:
vrc_results


# with open('numVRC/vrc_results.json', "w") as json_file:
#     json.dump(vrc_results,json_file)

In [ ]:
# vrc_results


# with open('numVRC/vrc_results.json', "w") as json_file:
#     json.dump(vrc_results,json_file)

with open('numVRC/vrc_results.json', "r") as json_file:
    vrc_results = json.load(json_file)

## Create elbow plots to determine appropriate number of clusters per context

In [ ]:
# text_column_name options: 'response_original' or 'merged_text'
# merged_text gives context by merging response with the scenario text and is preferred
text_column_name = 'merged_text'


# directory to place resulting plots and clusters
dir = "numFromElbow/"

# check that path exists and make it if it doesn't
if not os.path.exists(dir + 'elbow_plots/'):
    if not os.path.exists(dir):
            os.mkdir(dir)
    os.mkdir(dir + 'elbow_plots/')

# Define UMAP and clustering parameters
umap_params = {'n_neighbors': 100, 'n_components': 10, 'metric': 'cosine', 'min_dist': 0.05, 'random_state': None}

# creates plots and cluster_tables for each scenario
for scenario_number in range(1, 19):
    print("Running analysis on scenario " + str(scenario_number))
    # selects appropriate agent name based on scenario number
    agent_name = agent_list[scenario_number-1]
    words_to_remove = [agent_name, 'should', 'would', 'could']

    options_embeddings, options_df = compute_embeddings(df, text_column_name, scenario_number, words_to_remove)

    # elbow curve plots for determining appropriate number of clusters
    k_plot(options_embeddings, umap_params, f'{dir}elbow_plots/S{str(scenario_number)}.png')

## Run all scenarios in loop

In [ ]:
# Edit the values in this cell then run this cell and the one below

# directory to send resulting plots and clusters
dir = "numVRC/"

# text_column_name options: 'response_original' or 'merged_text'
# merged_text gives context by merging response with the scenario text and generally gives better results
text_column_name = 'merged_text'

# clusters_from options: 'elbow_results', 'same_as_manual', 'fixed'
# 'elbow_results' use the number of clusters for each context based on the results of the elbow plots (code below)
# 'same_as_manual' uses the same number of clusters for each context as was determine via manual coding for fair comparison between the two
# 'fixed' allows you to directly set the number of clusters
clusters_from = 'vrc'

if clusters_from == 'fixed':
    # Modify this if fixing the number of clusters
    numb_clusters = 3

# Define UMAP and clustering parameters
umap_params = {'n_neighbors': 100, 'n_components': 10, 'metric': 'cosine', 'min_dist': 0.05, 'random_state': None}

In [ ]:
# num of groups used for manual coding each scenario
num_manual_groups_by_scenario = [14, 16, 15, 16, 14, 15, 13, 15, 16, 14, 15, 17, 17, 18, 17, 14, 14, 14]

# optimal number of clusters for each scenario as determined by elbow plot analysis above
with open('numFromElbow/elbow_results.json', "r") as json_file:
    elbow_results = json.load(json_file)

In [ ]:
# check that path exists and make it if it doesn't
make_directories(dir)

# creates plots and cluster_tables for each scenario
for scenario_number in range(1, 19):
    print("Running analysis on scenario " + str(scenario_number))
    # selects appropriate agent name based on scenario number
    agent_list = ['Heinz', 'Josh', 'Brian', 'Liz', 'Mary', 'Brad', 'Darya', 'Eunice', 'Eamon', 'Cameron', 'Erica', 'Carl', 'Daniel', 'Andy', 'Ahmed', 'Eva', 'Jeff', 'Shania']
    agent_name = agent_list[scenario_number-1]


    words_to_remove = [agent_name, 'should', 'would', 'could']

    # Strip each response of words_to_remove and compute SBERT embeddings
    options_embeddings, options_df = compute_embeddings(df, text_column_name, scenario_number, words_to_remove)


    if(clusters_from == "elbow_results"):
        numb_clusters = elbow_results[str(scenario_number)]

    if(clusters_from == "vrc"):
        numb_clusters = vrc_results[str(scenario_number)]
        print("numb_clusters:", numb_clusters)

    # select appropriate number of clusters if the clusters are coming from 
    if clusters_from == 'same_as_manual':
        # + 1 is added to above number because some options were manually coded as 'other', giving one more category than the stated number
        numb_clusters = num_manual_groups_by_scenario[scenario_number-1]+1

    # define clustering params based on numb_clusters defined above
    clustering_params = {'algorithm_name': 'KMeans', 'num_clusters': numb_clusters}
    
    # compute clusters on embedded 
    umap_embeddings, cluster_assignment, clustering_model = compute_clusters(options_embeddings, clustering_params, umap_params)


    # print_clusters(options_df, text_column_name, cluster_assignment, numb_clusters)

    # create a df with clusters, centroids and umap embeddings
    clusters = compute_centroid(options_df, text_column_name, cluster_assignment, clustering_model, umap_embeddings)
    # reduce dimensionality to two for plotting
    umap_2(options_df, options_embeddings, clusters, umap_params)

    # options_df


    # save options_df for this scenario to cluster_tables
    options_df.to_csv(dir + 'cluster_tables/S' + str(scenario_number) + '.csv')
    # Call your function with the appropriate DataFrame columns
    plot_clusters(
        options_df['umap_dim2_0'],
        options_df['umap_dim2_1'],
        options_df['cluster'],
        # options_df["generation_number"],
        options_df["cluster_name"],
        options_df["response_original"],
        options_df["source"],
        umap_params,
        clustering_params,
        # id_gpt3=54,
        # id_human=36,
        f'{dir}plots/S{scenario_number}.html'
    )